In [1]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [2]:
!pip install -q toolz
!pip install -q wandb

In [3]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

In [4]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [5]:
csv = pd.read_csv("ZINC_310k.csv")

In [6]:
smiles = csv['SMILES']

In [7]:
smiles = smiles[:60000]

In [8]:
labels = torch.tensor(csv['LogP'][:60000]).float()

In [9]:
# labels = torch.tensor(csv['LogP']).float()

In [10]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [ ]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

 25%|██▌       | 3/12 [05:13<15:51, 105.72s/it]

In [10]:
# with open('saved/runner_20_logp_50_1_iter_1000.xml', 'rb') as f:
#     runner = pickle.load(f)

In [ ]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [ ]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [ ]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [ ]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])

X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [ ]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [ ]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [ ]:
torch.save(L_train, "L_train.pt")

torch.save(L_test, "L_test.pt")

torch.save(L_Val, "L_Val.pt")

In [ ]:
with open('train.npy', 'wb') as f:
    np.save(f, X_train)
    
with open('test.npy', 'wb') as f:
    np.save(f, X_test)
    
with open('validation.npy', 'wb') as f:
    np.save(f, X_Val)

In [22]:
# L_train = torch.load("L_train.pt")
# L_test = torch.load("L_test.pt")
# L_Val = torch.load("L_Val.pt")

In [14]:
# with open('train.npy', 'rb') as f:
#     X_train = np.load(f, allow_pickle=True)

# with open('test.npy', 'rb') as f:
#     X_test = np.load(f, allow_pickle=True)

# with open('validation.npy', 'rb') as f:
#     X_Val = np.load(f, allow_pickle=True)

In [33]:
print("Training model...")
runner.train_gen_pred(
    X_train,
    L_train,
    X_test,
    L_test,
    X_Val,
    L_Val,
    load_epoch= 0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=1000,
    batch_size=conf["batch_size"],
    num_workers=conf["num_workers"],
    label_pct=0.5
)

Training model...
Model #Params: 4732K


AttributeError: 'Adam' object has no attribute '_warned_capturable_if_run_uncaptured'

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)